Import packages and processed run metrics

In [19]:
# packages
import re
import numpy as np
import pandas as pd

# control
project_names = ['decay--pubmed']
validation_metric = 'Results/val_acc'

# read each data set
df = []
for project_name in project_names:
    temp = pd.read_csv(f'{project_name}.csv')
    print(temp.columns)
    df.append(temp)

df = pd.concat(df, axis=0, ignore_index=True)
#print(df.columns)

Index(['Unnamed: 0', 'dataset', 'method', 'finetune', 'n_epochs', 'lr_', 'lr',
       'beta', 'lr_finetune', 'regular_weight', 'K_finetune', 'K_',
       'regular_weight_finetune', 'regular_weight_', 'Results/test_acc',
       'Results/test_acc_bottom_decile', 'Results/test_acc_std',
       'Results/test_f1', 'Results/test_f1_bottom_decile',
       'Results/test_f1_std', 'Results/test_loss',
       'Results/test_loss_bottom_decile', 'Results/test_loss_std',
       'Results/val_acc', 'Results_unseen/test_acc',
       'Results_unseen/test_acc.1', 'Results_unseen/test_acc.2',
       'Results_unseen/test_acc.3', 'Results_unseen/test_f1',
       'Results_unseen/test_f1.1', 'Results_unseen/test_f1.2',
       'Results_unseen/test_f1.3', 'Results_unseen/test_loss',
       'Results_unseen/test_loss.1', 'Results_unseen/test_loss.2',
       'Results_unseen/test_loss.3', 'Results_weighted/test_acc',
       'Results_weighted/test_f1', 'Results_weighted/test_loss',
       'Results_weighted_unseen/te

Write function to return summaries of run's metric performance

In [10]:
def process_run_metrics(df, method='exact'):
    pass

    # get best runs for each group
    idx = df.groupby(['dataset', 'method', 'finetune']) \
        [validation_metric].idxmax()  # return index of max validation metric
    filtered_df = df.loc[idx]


    ## Process metrics and get top runs for each
    # are large or small metric values are desirable?
    descending_metrics = [name for name in filtered_df.columns if re.search('Results.*test', name)]
    ascending_metrics = [
        descending_metrics.pop(descending_metrics.index(name))
        for name in descending_metrics
        if re.search('std', name)
    ]

    # all non-metrics columns are used to identify the experimental run
    filtered_runs = filtered_df[[
        name for name in filtered_df.columns
        if name not in descending_metrics + ascending_metrics
    ]]

    # rank the metrics
    ranked_descending = filtered_df[descending_metrics].rank(
        method='first',
        ascending=False
    )
    ranked_ascending = filtered_df[ascending_metrics].rank(
        method='first',
        ascending=True
    )

    # combine and sort the ranked_metrics
    ranked_metrics = pd.concat([ranked_descending, ranked_ascending], axis=1)
    ranked_metrics = ranked_metrics[sorted(ranked_metrics.columns)]
    filtered_ranks = filtered_runs.join(ranked_metrics)


    ## Manipulate rank data to be summarized by runs and metrics
    # convert to long format
    # filter to top ranks
    long_filtered_ranks = pd.melt(filtered_ranks, id_vars=filtered_runs.columns, var_name='metric')
    top_filtered_metrics = long_filtered_ranks.loc[long_filtered_ranks.value <= 3].copy()  # top 3 runs

    # compute rank summaries to understand what runs are top overall
    top_filtered_metrics['rank_one_ind'] = (top_filtered_metrics.value == 1)
    top_filtered_metrics['rank_two_ind'] = (top_filtered_metrics.value == 2)
    top_filtered_metrics['rank_three_ind'] = (top_filtered_metrics.value == 3)
    top_filtered_metrics.replace(False, pd.NA, inplace=True)

    # summarized metric ranks for run type
    rank_summary_columns = ['rank_one_ind', 'rank_two_ind', 'rank_three_ind', 'value']
    id_columns = ['dataset', 'method', 'finetune']
    run_summary = top_filtered_metrics.groupby(id_columns)[rank_summary_columns].count()

    # summarize metric ranks for metric choice
    metric_summary = top_filtered_metrics.loc[top_filtered_metrics.method == method]
    metric_summary = metric_summary.sort_values(by='metric').groupby('metric')[rank_summary_columns].count()

    return (
        metric_summary,
        run_summary,
        filtered_runs
    )


Filter data to runs of interest
Apply summary function to filtered dataset

In [11]:
subset_df = df
subset_df = subset_df.loc[(df.n_epochs < 6)]
#subset_df = subset_df.loc[(df.beta * 10 % 2 == 0) | (df.method != 'exact')]
(my_metric_summary, my_run_summary, _) = process_run_metrics(subset_df, method='exact')

C:\Users\joela\AppData\Local\Temp\ipykernel_27484\1571230217.py:26: FutureWarning: Dropping of nuisance columns in DataFrame.rank is deprecated; in a future version this will raise TypeError. Select only valid columns before calling rank.
  ranked_descending = filtered_df[descending_metrics].rank(


View processed summaries

In [12]:
print(my_metric_summary.shape)
sort_by = ['value', 'rank_one_ind', 'rank_two_ind', 'rank_three_ind']
my_metric_summary.sort_values(by=sort_by, ascending=False)

(7, 4)


,rank_one_ind,rank_two_ind,rank_three_ind,value
metric,,,,
Results_unseen/test_acc_std,0,1,0,1
Results_unseen/test_loss_bottom_decile,0,1,0,1
Results_unseen/test_loss_std,0,1,0,1
Results/test_f1,0,0,1,1
Results_unseen/test_acc_bottom_decile,0,0,1,1
Results_unseen/test_f1_std,0,0,1,1
Results_weighted/test_f1,0,0,1,1


In [13]:
print(my_run_summary.shape)
my_run_summary.sort_values(by='value', ascending=False)

(19, 4)


rank_one_ind  rank_two_ind  rank_three_ind  value
dataset method finetune                                                   
pubmed  fedbn  1                    7             1               2     10
        ditto  1                    1             3               5      9
        fedavg 1                    1             4               4      9
        pfedme 1                    1             4               3      8
sst2    pfedme 0                    3             0               2      5
        exact  0                    0             3               2      5
pubmed  fedbn  0                    2             3               0      5
sst2    ditto  0                    4             0               0      4
pubmed  ditto  0                    3             0               1      4
        fedavg 0                    1             3               0      4
        fedem  1                    2             1               0      3
               0                    1             0               2      3
sst2    ditto  1                    0             2               0      2
        fedbn  0                    0             0               2      2
        fedem  0                    0             1               1      2
pubmed  exact  1                    0             0               2      2
sst2    pfedme 1                    0             2               0      2
        fedavg 0                    0             0               1      1
        fedbn  1                    1             0               0      1